<a href="https://colab.research.google.com/github/jddfrance/OnTheIssue_Scrapper/blob/main/Ontheissue_opnions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from fuzzywuzzy import process
import lxml.html as lh

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#Making the URL list of HouseMatchs from the OnTheIssues website

#Get list of states abreviations
s_list = pd.read_html('https://www.ssa.gov/international/coc-docs/states.html')[0][1].tolist()
s_list.append("house")

#get list of working folders on the website
b_url = "https://www.ontheissues.org/"
responses = []
for state in s_list:
    url = f'{b_url}{state}/'
    response = requests.get(url)
    if response.reason == "OK":
        responses.append(url)

    elif response.reason != "Not Found":
        raise ValueError("The error isn't 202 or 404, attention required")

#Extract all the HouseMatch pages for candidates
housematch_list = []
for folder in responses:
    reqs = requests.get(folder)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    #get links that have HouseMatch and create a list with the complete link
    for link in soup.find_all('a', string = re.compile('HouseMatch', re.IGNORECASE)):
        housematch_list.append("https://www.ontheissues.org" + link.get('href'))


In [ ]:
name_list = []
for link in housematch_list:
  name = link.split('/')[-1].replace('_HouseMatch.htm', '').replace('_', ' ')
  name_list.append(name)

In [ ]:
#filter based on candidates list from NGO
NGO = pd.read_excel("NGO.xlsx")

#Match candidate names with housematch links
#Execution time of string matching: 50m on google collab
NGO_fill = []
for index, row in NGO.iterrows():
    str2Match = row["name"]
    strOptions = housematch_list
    #Select the string with the highest matching percentage
    highest = process.extractOne(str2Match,strOptions)
    if highest[1] > 50:
        row = {"Name": row["name"], "party": row["party"], "state": row["state"],
               "district": row["district"], "level": row["level"], "HouseMatch": highest[0],
               "Aprox": highest[1]}
        NGO_fill.append(row)


OTI_urls = pd.DataFrame(NGO_fill)
OTI_urls.to_excel("OTI_urls.xlsx")

In [ ]:
OTI_urls = pd.read_excel("OTI_urls.xlsx").drop(columns= 'Unnamed: 0')
OTI_urls = OTI_urls[OTI_urls["Aprox"] > 71].reset_index()

In [ ]:
OTI_urls

,index,Name,party,state,district,level,HouseMatch,Aprox
0,0,Jerry Carl,Republican,Alabama,1,house,https://www.ontheissues.org/house/Jerry_Carl_H...,81
1,1,Barry Moore,Republican,Alabama,2,house,https://www.ontheissues.org/house/Barry_Moore_...,82
2,3,Mike Rogers,Republican,Alabama,3,house,https://www.ontheissues.org/MI/Mike_Rogers_Hou...,82
3,5,Robert Aderholt,Republican,Alabama,4,house,https://www.ontheissues.org/house/Robert_Aderh...,84
4,14,Gary Palmer,Republican,Alabama,6,house,https://www.ontheissues.org/house/Gary_Palmer_...,82
...,...,...,...,...,...,...,...,...
348,1548,Carol Miller,Republican,West Virginia,1,house,https://www.ontheissues.org/house/Carol_Miller...,83
349,1552,David McKinley,Republican,West Virginia,2,house,https://www.ontheissues.org/house/David_McKinl...,84
350,1553,Alex Mooney,Republican,West Virginia,2,house,https://www.ontheissues.org/house/Alex_Mooney_...,82
351,1556,Mark Pocan,Democratic,Wisconsin,2,house,https://www.ontheissues.org/house/Mark_Pocan_H...,81


In [ ]:
#Example of topics and positions from one HouseMatch

,Theme,Position
1,topic 1:Abortion is a woman's unrestricted right,Strongly Opposes
2,topic 2:Legally require hiring women & minori...,Opposes
3,topic 3:Comfortable with same-sex marriage,Strongly Opposes
4,topic 4:Keep God in the public sphere,Strongly Favors
5,topic 5:Expand ObamaCare,Strongly Opposes
6,topic 6:Privatize Social Security,No opinion on
7,topic 7:Vouchers for school choice,No opinion on
8,topic 8:Fight EPA regulatory over-reach,No opinion on
9,topic 9:Stricter punishment reduces crime,Strongly Favors
10,topic 10:Absolute right to gun ownership,Favors


In [ ]:
#Some questions have inverse answers compared to their counterparts, this is to fix that
#NA means Not available
def inverse_p(position):
    if position == "Strongly Opposes":
        return "Strongly Favors"
    elif position == "Opposes":
        return "Favors"
    elif position == "Strongly Favors":
        return "Strongly Opposes"
    elif position == "Favors":
        return "Opposes"
    elif position == "No opinion on" or position == "Neutral on":
        return "Neutral"
    elif position == "NA":
        return position
    else:
        print(row["HouseMatch"]) #use is specific for the loop
        raise ValueError('The Value is not a valid position')

#function for readability and troubleshooting       
def get_topic(num, lista):
    try:
        return lista[(num-1)]
    except:
        print(row["HouseMatch"]) #use is specific for the loop
        raise ValueError('List has some problem')

#function for Key Issues
def key_i(position):
    if position == "Strongly Opposes" or position == "Strongly Favors":
        return "T"
    elif position == "not found"  or position == "NA":
        return "NA"    
    else:
        return "F"        

In [ ]:
OTI_urls

,index,Name,party,state,district,level,HouseMatch,Aprox
0,0,Jerry Carl,Republican,Alabama,1,house,https://www.ontheissues.org/house/Jerry_Carl_H...,81
1,1,Barry Moore,Republican,Alabama,2,house,https://www.ontheissues.org/house/Barry_Moore_...,82
2,3,Mike Rogers,Republican,Alabama,3,house,https://www.ontheissues.org/MI/Mike_Rogers_Hou...,82
3,5,Robert Aderholt,Republican,Alabama,4,house,https://www.ontheissues.org/house/Robert_Aderh...,84
4,14,Gary Palmer,Republican,Alabama,6,house,https://www.ontheissues.org/house/Gary_Palmer_...,82
...,...,...,...,...,...,...,...,...
348,1548,Carol Miller,Republican,West Virginia,1,house,https://www.ontheissues.org/house/Carol_Miller...,83
349,1552,David McKinley,Republican,West Virginia,2,house,https://www.ontheissues.org/house/David_McKinl...,84
350,1553,Alex Mooney,Republican,West Virginia,2,house,https://www.ontheissues.org/house/Alex_Mooney_...,82
351,1556,Mark Pocan,Democratic,Wisconsin,2,house,https://www.ontheissues.org/house/Mark_Pocan_H...,81


In [ ]:
row_list = []
for index, row in OTI_urls.iterrows():
    #get table elements from page
    url = row["HouseMatch"]
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    #Topic text will be used to store the strings that contain the topics
    topic_text = []
    for element in tr_elements:
        topic_text.append(element[0].text_content())

    #Creating a list to search and organize the topics
    topic_list = []
    for i in range(20):
        topic_list.append(f"topic {i+1}:")

    #Searching and organizing topics by order
    topic_ord = []
    topics_wr = 0
    for substring in topic_list:
        for bigstring in topic_text:
            if substring in bigstring:
                topic_ord.append(bigstring)
                topics_wr += 1
        #When the topic is not found, append a topic X not found
        if topics_wr >= 1 and topic_list.index(substring)+1 != len(topic_ord):
            topic_ord.append(f"{substring} not found")
            topics_wr += 1

    #Extract opnions from text
    posi_list = []
    valid_opnions = ["Neutral on", "No opinion on", 
                     "Strongly Opposes", "Strongly Favors", 
                     "Opposes", "Favors", "NA", "not found"]
    for topic in topic_ord:
        tsplit = topic.split("\r\n")
        if len(tsplit) > 2:
            position = tsplit[2]
            #check if this is a opnion and if it's not, try another element of the list based on the exploration of website 
            if any(position in s for s in valid_opnions):
                posi_list.append(position)
            elif any(tsplit[1] in s for s in valid_opnions):
                posi_list.append(tsplit[1])
            else:
                print(row["HouseMatch"])
                raise ValueError('Attention needed here, table outside identified pattern')

        elif len(tsplit) == 1 and tsplit[0].endswith("not found"):
            posi_list.append("NA")
        else:
            print(row["HouseMatch"])
            raise ValueError('Attention needed here, table outside identified pattern')


    #Get values from list and convert to NGO answers
    Q1 = "NA"
    Q2 = get_topic(5, posi_list)
    Q3 = inverse_p(get_topic(7, posi_list)) 
    Q4 = get_topic(20, posi_list)
    Q5 = inverse_p(get_topic(12, posi_list)) 
    Q6 = inverse_p(get_topic(9, posi_list)) 
    Q7 = "NA"
    Q8 = get_topic(2, posi_list)
    Q9 = inverse_p(get_topic(13, posi_list)) 
    Q10 = inverse_p(get_topic(8, posi_list)) 
    Q11 = inverse_p(get_topic(19, posi_list)) 
    Q12 = inverse_p(get_topic(15, posi_list)) 
    Q13 = inverse_p(get_topic(11, posi_list)) 
    Q14 = get_topic(10, posi_list)
    Q15 = get_topic(1, posi_list)
    Q16 = "NA"

    #Row for Q_answers in the final dataframe 
    row = {"name": row['Name'], "party": row["party"], "state": row["state"],
           "district": row["district"], "level": row["level"], 
           "Accuracy": row['Aprox'], "Source": row["HouseMatch"],
           "q1 answer": Q1, "q2 answer": Q2,"q3 answer": Q3,
           "q4 answer": Q4,"q5 answer": Q5,"q6 answer": Q6,
           "q7 answer": Q7,"q8 answer": Q8,"q9 answer": Q9,
           "q10 answer": Q10,"q11 answer": Q11,"q12 answer": Q12,
           "q13 answer": Q13,"q14 answer": Q14,"q15 answer": Q15,
           "q16 answer": Q16, "q1 key issue": key_i(Q1), "q2 key issue": key_i(Q2), 
           "q3 key issue": key_i(Q3), "q4 key issue": key_i(Q4), "q5 key issue": key_i(Q5), 
           "q6 key issue": key_i(Q6), "q7 key issue": key_i(Q7), "q8 key issue": key_i(Q8), 
           "q9 key issue": key_i(Q9), "q10 key issue": key_i(Q10), "q11 key issue": key_i(Q11), 
           "q12 key issue": key_i(Q12), "q13 key issue": key_i(Q13), "q14 key issue": key_i(Q14), 
           "q15 key issue": key_i(Q15), "q16 key issue": key_i(Q16)}
    
    
    row_list.append(row)

In [ ]:
NGO_qans = pd.DataFrame(row_list)

In [ ]:
NGO_qans.to_excel("NGO_parcial.xlsx")

In [ ]:
for column in NGO_qans.columns[7:23]:
    NGO_qans.loc[NGO_qans[column] == "Strongly Opposes", column] = "SD"
    NGO_qans.loc[NGO_qans[column] == "Opposes", column] = "D"
    NGO_qans.loc[NGO_qans[column] == "Neutral on", column] = "N"
    NGO_qans.loc[NGO_qans[column] == "Neutral", column] = "N"
    NGO_qans.loc[NGO_qans[column] == "No opinion on", column] = "N"
    NGO_qans.loc[NGO_qans[column] == "Favors", column] = "A"
    NGO_qans.loc[NGO_qans[column] == "Strongly Favors", column] = "SA"

In [ ]:
NGO_qans.to_excel("NGO_final.xlsx", index=False)